In [2]:
import pandas as pd
import numpy as np
from sklearn.decomposition import TruncatedSVD

In [3]:
# ----------------------
# 1. 원본 게임 데이터 로드
# ----------------------
df = pd.read_csv("popular_games.csv")

games = df[["game_id", "game_name", "playtime", "rating", "genre", "owned_ratio"]].copy()

# ----------------------
# 2. 선택 확률 계산 (owned_ratio + rating)
# ----------------------
ratio_norm  = (games["owned_ratio"] - games["owned_ratio"].min()) / (games["owned_ratio"].max() - games["owned_ratio"].min())
rating_norm = (games["rating"] - games["rating"].min()) / (games["rating"].max() - games["rating"].min())
score = 0.6 * ratio_norm + 0.4 * rating_norm

# softmax 변환
p = np.exp(score - score.max())
p = p / p.sum()

In [4]:
# ----------------------
# 3. 가상 유저-게임 상호작용 생성
# ----------------------
n_users = 100   # 유저 수
k = 5           # 각 유저가 선택하는 게임 수

interactions = []
rng = np.random.default_rng(42)

for u in range(n_users):
    chosen = rng.choice(games["game_id"].values, size=k, replace=False, p=p)
    for g in chosen:
        row = games.loc[games["game_id"] == g]
        base_rating = row["rating"].values[0]
        base_ratio  = row["owned_ratio"].values[0]
        # rating + owned_ratio 결합 점수
        value = 0.7 * base_rating + 0.3 * base_ratio
        interactions.append((f"user_{u}", g, value))   # user 이름은 "user_0", "user_1", ...

interactions = pd.DataFrame(interactions, columns=["user_name", "game_id", "score"])

print("가상데이터 예시:")
print(interactions.head())

# ----------------------
# 4. 유저-게임 행렬 생성
# ----------------------
mat = interactions.pivot_table(index="user_name", columns="game_id", values="score", aggfunc="mean").fillna(0.0)


가상데이터 예시:
  user_name  game_id  score
0    user_0   665016  2.933
1    user_0    12895  3.088
2    user_0    12606  2.905
3    user_0   966102  2.774
4    user_0   442840  3.124


In [5]:
# ----------------------
# 5. SVD 기반 추천 함수
# ----------------------
def svd_recommend(user_name, top_n=5, n_components=10):
    svd = TruncatedSVD(n_components=min(n_components, min(mat.shape)-1), random_state=42)
    U = svd.fit_transform(mat.values)
    Vt = svd.components_
    recon = np.dot(U, Vt)
    
    # user index 찾기
    user_idx = list(mat.index).index(user_name)
    user_actual = mat.values[user_idx]
    user_pred = recon[user_idx]
    
    # 안 본 게임만 추천
    recommendations = {
        gid: pred for gid, pred in zip(mat.columns, user_pred) if user_actual[mat.columns.get_loc(gid)] == 0
    }
    top_games = sorted(recommendations, key=recommendations.get, reverse=True)[:top_n]
    
    # game_id → name 변환
    game_names = games.set_index("game_id").loc[top_games, "game_name"].values.tolist()
    return game_names

In [6]:
# 6. 추천 예시
# ----------------------
print("\n추천 결과 for user_0:", svd_recommend("user_0", top_n=5))


추천 결과 for user_0: ['Delta Force', 'Awaria', 'We Were Here', 'Ghost Master', 'Streets of Rogue']
